### ライブラリのインポートと必要な関数の定義

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import random
import logging
from pathlib import Path
from tqdm import tqdm

from imageSearch.DatabaseManager import FAISSDatabaseManager
from imageSearch.ImageFeatureExtractor import Resnet50ImageFeatureExtractor, ONNXImageFeatureExtractor, DreamSimImageFeatureExtractor
from imageSearch.utils.display_notebook import display_images_grid_html
from imageSearch.utils.logger_util import configure_logger

logger = configure_logger()
logger.setLevel(logging.INFO)
# logger.setLevel(logging.ERROR)


def register_images(image_dir, db_manager, extractor):
    image_dir = Path(image_dir)
    image_paths = [
        str(p) for p in image_dir.glob("**/*")
        if p.suffix.lower() in [".jpg", ".jpeg", ".png"]
    ]
    rows = []
    for img_path in tqdm(image_paths, desc="Registering images"):
        try:
            feature = extractor.extract_feature(img_path)
            rows.append({"embedding": feature, "file_path": img_path})
        except Exception as e:
            logging.error(f"Error processing {img_path}: {e}")
    db_manager.insert_embeddings(rows)

def search_images(db_manager, extractor, query_image_path, k=5):
    # 特徴抽出
    query_feature = extractor.extract_feature(query_image_path)
    # FAISSによる検索
    distances, indices = db_manager.search(query_feature, k)
    results = []
    for d, idx in zip(distances[0], indices[0]):
        # インデックスが-1の場合は該当なしとする
        if idx != -1:
            file_path = db_manager.file_paths[idx]
            results.append((d, file_path))
    return results


c:\Users\hiahara\miniforge3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### データベースの初期化・画像の登録


In [2]:
# 画像特徴抽出器のインスタンス生成
# extractor = Resnet50ImageFeatureExtractor()
# extractor = DreamSimImageFeatureExtractor(cache_dir="./model/DreamSim/")
extractor = ONNXImageFeatureExtractor(onnx_path="./model/ONNX/mobilenet_v2.onnx")

# FAISSの設定
print(extractor.dim)
db_manager = FAISSDatabaseManager(dim = extractor.dim, recreate=True)

# # 画像ディレクトリから画像登録
image_directory = "../../sample_data/"
register_images(image_directory, db_manager, extractor)


2025-02-11 03:56:38,179 - imageSearch - INFO - ONNX model input: name=input.1, shape=[1, 3, 224, 224], type=tensor(float)
2025-02-11 03:56:38,179 - imageSearch - INFO - モデルの出力次元: 1000
2025-02-11 03:56:38,194 - imageSearch - INFO - FAISSでは接続処理は不要です。
2025-02-11 03:56:38,195 - imageSearch - INFO - 新しいFAISSインデックスを作成しました。


1000


Registering images:   0%|          | 0/206 [00:00<?, ?it/s]2025-02-11 03:56:38,330 - imageSearch - ERROR - 画像の読み込みに失敗しました。path: ..\..\sample_data\corrupted_image\sample_corrupted_jpg.jpg error: cannot identify image file '..\\..\\sample_data\\corrupted_image\\sample_corrupted_jpg.jpg'
ERROR:root:Error processing ..\..\sample_data\corrupted_image\sample_corrupted_jpg.jpg: cannot identify image file '..\\..\\sample_data\\corrupted_image\\sample_corrupted_jpg.jpg'
Registering images:   2%|▏         | 5/206 [00:00<00:05, 37.38it/s]2025-02-11 03:56:38,331 - imageSearch - ERROR - 画像の読み込みに失敗しました。path: ..\..\sample_data\corrupted_image\sample_png.png error: cannot identify image file '..\\..\\sample_data\\corrupted_image\\sample_png.png'
ERROR:root:Error processing ..\..\sample_data\corrupted_image\sample_png.png: cannot identify image file '..\\..\\sample_data\\corrupted_image\\sample_png.png'
Registering images: 100%|██████████| 206/206 [00:02<00:00, 84.12it/s]
2025-02-11 03:56:40,648 - imag

### 検索の実行

In [3]:
# クエリ画像を用いた検索
query_image = str(random.choice(list(Path("../../sample_data/").glob("**/*.jpg"))))
k = 5
results = search_images(db_manager, extractor, query_image, k=k)

# 検索結果の表示
if results:
    for distance, file_path in results:
        print(f"Distance: {distance:.4f}, File: {file_path}")
else:
    print("検索結果が見つかりませんでした。")
    

images = [query_image] + ["" for _ in range(k-1)] + [r[1] for r in results]
labels = [f"{Path(query_image).name}"] + ["" for _ in range(k-1)] + [f"{Path(r[1]).name}\n(Distance: {r[0]:.05})" for r in results]
display_images_grid_html(images, labels=labels, cols=k)

Distance: 1.0000, File: ..\..\sample_data\WIDER_OpenData\2--Demonstration\2_Demonstration_Demonstration_Or_Protest_2_76.jpg
Distance: 0.6421, File: ..\..\sample_data\WIDER_OpenData\8--Election_Campain\8_Election_Campain_Election_Campaign_8_25.jpg
Distance: 0.6057, File: ..\..\sample_data\WIDER_OpenData\7--Cheering\7_Cheering_Cheering_7_134.jpg
Distance: 0.5968, File: ..\..\sample_data\WIDER_OpenData\7--Cheering\7_Cheering_Cheering_7_118.jpg
Distance: 0.5929, File: ..\..\sample_data\WIDER_OpenData\4--Dancing\4_Dancing_Dancing_4_53.jpg


2_Demonstration_Demonstration_Or_Protest_2_76.jpg,,,,
2_Demonstration_Demonstration_Or_Protest_2_76.jpg(Distance: 1.0),8_Election_Campain_Election_Campaign_8_25.jpg(Distance: 0.64211),7_Cheering_Cheering_7_134.jpg(Distance: 0.60571),7_Cheering_Cheering_7_118.jpg(Distance: 0.59676),4_Dancing_Dancing_4_53.jpg(Distance: 0.59291)


### FAISSインデックスの保存

In [4]:
db_manager.save(save_dir=str("./localDB/FAISS/sampleDB/"), file_name="sampleDB.index")

2025-02-11 03:56:40,946 - imageSearch - INFO - FAISSインデックスをファイルに保存しました: localDB\FAISS\sampleDB\sampleDB.index
2025-02-11 03:56:40,947 - imageSearch - INFO - 画像ファイルパスをファイルに保存しました: localDB\FAISS\sampleDB\sampleDB.pkl
